<a href="https://colab.research.google.com/github/daypick/katago-colab/blob/master/katago_1_8_latest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun May 16 07:14:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

This document shows how to run KataGo on Colab, and how to connect it using `Sabaki`, `Lizzie` (or other GTP engine supported apps) in your local machine.

In [ ]:
NGROK_TOKEN="1ra0YWxwAEUSKnPLTsOnCL0vBhr_6F97AL6M5hL5Mgxuk97VA"
USER_NAME="root"
USER_PASSWORD="latest"

# optional args
# supports: OPENCL, CUDA or AUTO
KATAGO_BACKEND="CUDA"
# supports: 40b, 20b or AUTO
WEIGHT_FILE="40b" 

import subprocess
gpu_name=str(subprocess.check_output("nvidia-smi -q | grep \"Product Name\" | cut -d\":\" -f2 | tr -cd '[:alnum:]._-'", shell=True), encoding='utf-8')
if KATAGO_BACKEND == "AUTO":
  if gpu_name == "TeslaT4":
    KATAGO_BACKEND="CUDA"
  else:
    KATAGO_BACKEND="OPENCL"

if WEIGHT_FILE == "AUTO":
  if gpu_name == "TeslaK80" or gpu_name == "TeslaP4":
    WEIGHT_FILE="20b"
  else:
    WEIGHT_FILE="40b"

!echo "Using Katago Backend : " $KATAGO_BACKEND
!echo "Using Katago Weight : " $WEIGHT_FILE
!echo "GPU : " $gpu_name

# Install useful stuff
!apt-get update 1>/dev/null
!apt install --yes ssh screen nano htop ranger git libzip4 1>/dev/null
!pip install oss2 1>/dev/null
!wget -q https://github.com/wonsiks/katago-colab/releases/download/v1.8.0/libzip.so.5.0 -O /usr/lib/x86_64-linux-gnu/libzip.so.5
%cd /content

!rm -rf katago-colab
!git clone https://github.com/kinfkong/katago-colab.git 1>/dev/null

#download the binarires
!wget -q https://github.com/wonsiks/katago-colab/releases/download/v1.8.2/katago-$KATAGO_BACKEND -O katago
!chmod +x /content/katago
!wget -q https://github.com/kinfkong/katago-colab/releases/download/v1.4.5/ngrok -O ngrok
!chmod +x /content/ngrok
!mkdir -p /root/.katago/
!cp -r /content/katago-colab/opencltuning /root/.katago/


################################################################################################
#############download the weights #######################
#weight_urls = {
#    '40b': 'https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b40c256-s8003455744-d1941206379.bin.gz',
#    '40n': 'https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b40c256-s8049900800-d1952632576.bin.gz'
#}

#weight_url = weight_urls[WEIGHT_FILE]
#!wget --quiet $weight_url -O $WEIGHT_FILE".bin.gz" 
#!chmod +x $WEIGHT_FILE".bin.gz"
#!rm -rf weight.bin.gz
#!ln -s $WEIGHT_FILE".bin.gz" weight.bin.gz

################################################################################################
#download the weights
#previous strongest:=> https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b40c256-s8096761856-d1963946463.bin.gz
#srong=>  https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b40c256-s8003455744-d1941206379.bin.gz
#latest=> https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b40c256-s8049900800-d1952632576.bin.gz
#most-strong=> https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b40c256-s8003455744-d1941206379.bin.gz
#second-strong https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b40c256-s8026817280-d1946950232.bin.gz

!rm -rf weight.bin.gz

if WEIGHT_FILE == "40b" :
  !wget -q https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b40c256-s8073397504-d1958161418.bin.gz -O weight.bin.gz
#  KATAGO_URL=open("kataurl.txt", mode='r').read()
#  !wget -q $KATAGO_URL -O weight.bin.gz
else:
  !wget -q https://media.katagotraining.org/uploaded/networks/models/kata1/kata1-b40c256-s8049900800-d1952632576.bin.gz -O weight.bin.gz

# SSH setting
!echo "root:$USER_PASSWORD" | chpasswd
!echo "PasswordAuthentication yes" > /etc/ssh/sshd_config
!echo "PermitUserEnvironment yes" >> /etc/ssh/sshd_config
!echo "PermitRootLogin yes" >> /etc/ssh/sshd_config

!mkdir -p /root/.ssh

# generate the keys
#!ssh-keygen -q -t rsa -N '' -f /root/.ssh/id_rsa <<<y 2>&1 >/dev/null
#!cat /root/.ssh/id_rsa.pub >> /root/.ssh/authorized_keys

!service ssh restart > /dev/null

# Run ngrok
get_ipython().system_raw('./ngrok authtoken $NGROK_TOKEN && ./ngrok tcp 22 &')
!sleep 5

import oss2
import requests
import json
from re import sub
r = requests.get('http://localhost:4040/api/tunnels')
raw_ssh = r.json()['tunnels'][0]['public_url']
ssh_args = (sub("tcp://", "", raw_ssh)).split(':')
ssh_option = {
    'host': ssh_args[0],
    'port': int(ssh_args[1]),
    'user': 'root'
}

ssh_option_json = json.dumps(ssh_option)

endpoint = 'http://oss-cn-beijing.aliyuncs.com'
auth = oss2.Auth('LTAI4G3tthX2R3Z8KY6Bdbs3', 'Q22UstMdKX8zZY9BqkGWWd2XbEBvPH')
bucket = oss2.Bucket(auth, endpoint, 'kata-config')

key = USER_NAME + '.ssh.json'
bucket.put_object(key, ssh_option_json)

if KATAGO_BACKEND == "CUDA":
  %cd /usr/lib/x86_64-linux-gnu/
  !wget -q https://github.com/wonsiks/katago-colab/releases/download/v1.8.0/libcublas.so.11.3.0.106 -O libcublas.so.11
  !wget -q https://github.com/wonsiks/katago-colab/releases/download/v1.8.0/libcublasLt.so.11.3.0.106 -O libcublasLt.so.11
  %cd /content
  !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/libcudnn8_8.0.5.39-1+cuda11.1_amd64.deb
  !dpkg --configure -a
  !dpkg -i libcudnn8_8.0.5.39-1+cuda11.1_amd64.deb
else:
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaT4_x19_y19_c256_mv10.txt -O /root/.katago/opencltuning/tune8_gpuTeslaT4_x19_y19_c256_mv10.txt
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaT4_x19_y19_c256_mv8.txt -O /root/.katago/opencltuning/tune8_gpuTeslaT4_x19_y19_c256_mv8.txt
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaP4_x19_y19_c256_mv10.txt -O /root/.katago/opencltuning/tune8_gpuTeslaP4_x19_y19_c256_mv10.txt
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaK80_x19_y19_c256_mv10.txt -O /root/.katago/opencltuning/tune8_gpuTeslaK80_x19_y19_c256_mv10.txt
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaK80_x19_y19_c256_mv8.txt -O /root/.katago/opencltuning/tune8_gpuTeslaK80_x19_y19_c256_mv8.txt
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaP100PCIE16GB_x19_y19_c256_mv10.txt -O /root/.katago/opencltuning/tune8_gpuTeslaP100PCIE16GB_x19_y19_c256_mv10.txt
  !wget -q https://raw.githubusercontent.com/wonsiks/katago-colab/master/opencltuning/tune8_gpuTeslaP100PCIE16GB_x19_y19_c256_mv8.txt -O /root/.katago/opencltuning/tune8_gpuTeslaP100PCIE16GB_x19_y19_c256_mv8.txt
!wget https://github.com/wonsiks/katago-colab/releases/download/v1.8.0/gtp_colab.cfg
!rm -rf /content/gtp_colab.cfg* && /content/kataurl.txt*
!wget -q  -O /content/katago-colab/config/gtp_colab.cfg

!echo -e "\n[KataGo Config]" 
!cat /content/katago-colab/config/gtp_colab.cfg
!echo -e "\n"

!echo "Done! Now you can connect..."
!echo "Engine command for Lizzie : ./colab-katago.exe "$USER_NAME $USER_PASSWORD



Using Katago Backend :  CUDA
Using Katago Weight :  40b
GPU :  TeslaV100-SXM2-16GB


E: Unmet dependencies. Try 'apt --fix-broken install' with no packages (or specify a solution).
/content
Cloning into 'katago-colab'...
remote: Enumerating objects: 102, done.
remote: Total 102 (delta 0), reused 0 (delta 0), pack-reused 102
Receiving objects: 100% (102/102), 32.42 KiB | 4.63 MiB/s, done.
Resolving deltas: 100% (46/46), done.
ngrok: Text file busy
/usr/lib/x86_64-linux-gnu
/content
--2021-05-16 07:14:49--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1604/x86_64/libcudnn8_8.0.5.39-1+cuda11.1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.199.20.126
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.199.20.126|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 506495158 (483M) [application/x-deb]
Saving to: ‘libcudnn8_8.0.5.39-1+cuda11.1_amd64.deb.6’

libcudnn8_8.0.5.39- 100%[=

#The following sections are for debugging only, you can ignore.

**Shows the Colab GPU Info**

Get your ssh login info
*ssh login account is root, the login password is the `USER_PASSWORD` you configured in the previous steps*

In [ ]:
import requests
from re import sub
r = requests.get('http://localhost:4040/api/tunnels')
str_ssh = r.json()['tunnels'][0]['public_url']
str_ssh = sub("tcp://", "", str_ssh)
str_ssh = sub(":", " -p ", str_ssh)
str_ssh = "ssh root@" + str_ssh 
print(str_ssh)

ssh root@6.tcp.ngrok.io -p 15502


In [ ]:
!./katago benchmark -model weight.bin.gz -config katago-colab/config/gtp_colab.cfg

2021-05-16 07:15:39+0000: Loading model and initializing benchmark...
2021-05-16 07:15:39+0000: nnRandSeed0 = 17687316231351245371
2021-05-16 07:15:39+0000: After dedups: nnModelFile0 = weight.bin.gz useFP16 auto useNHWC auto
2021-05-16 07:15:41+0000: Cuda backend thread 0: Found GPU Tesla V100-SXM2-16GB memory 16945512448 compute capability major 7 minor 0
2021-05-16 07:15:41+0000: Cuda backend thread 0: Model version 10 useFP16 = true useNHWC = true
2021-05-16 07:15:41+0000: Cuda backend thread 0: Model name: kata1-b40c256-s8096761856-d1963946463

2021-05-16 07:15:43+0000: Loaded config katago-colab/config/gtp_colab.cfg
2021-05-16 07:15:43+0000: Loaded model weight.bin.gz

Testing using 800 visits.
  If you have a good GPU, you might increase this using "-visits N" to get more accurate results.
  If you have a weak GPU and this is taking forever, you can decrease it instead to finish the benchmark faster.

Your GTP config is currently set to cudaUseFP16 = auto and cudaUseNHWC = auto


In [ ]:
!./katago genconfig -model weight.bin.gz -output gtp_custom.cfg

File gtp_custom.cfg already exists, okay to overwrite it with an entirely new config (y/n)?
^C


In [ ]:
!./katago version

KataGo v1.8.2
Git revision: b846bddd88fbc5353e4a93fa514f6cbf45358362
Compile Time: Apr 18 2021 16:55:08
Using CUDA backend
Compiled with CUDA version 11.1.105
Compiled to support contributing to online distributed selfplay
